# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Find and reference Output File (CSV) from Weather analysis
output_file = "output_data/CityWeath.csv"

# pull prevously generated and saved output data from csv
CityWeath_df = pd.read_csv(output_file)

CityWeath_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,1337610,Thinadhoo,MV,0.5333,72.9333,83.25,69,98,15.64
1,6185377,Yellowknife,CA,62.4560,-114.3525,24.67,100,90,4.61
2,4035715,Avarua,CK,-21.2078,-159.7750,73.45,73,20,6.91
3,2138285,Tadine,NC,-21.5500,167.8833,71.82,66,19,5.93
4,5848280,Kapaa,US,22.0752,-159.3190,73.58,78,1,1.01
...,...,...,...,...,...,...,...,...,...
685,3444105,San Matías,BO,-16.3667,-58.4000,69.31,85,75,2.77
686,3870243,Taltal,CL,-25.4000,-70.4833,59.13,79,28,1.19
687,2063056,Port Augusta,AU,-32.5000,137.7667,64.71,61,8,12.48
688,5866063,Kenai,US,60.5544,-151.2583,27.46,93,90,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Use airport_heatmap exercise as base
# Configure gmaps
gmaps.configure(api_key=g_key)

# # Store latitude and longitude in locations
# locations = CityWeath_df[["latitude", "longitude"]]

In [4]:
# Use airport_heatmap exercise as base
# Store latitude and longitude in locations
locations = CityWeath_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
rating = CityWeath_df["Humidity"].astype(float)

In [6]:
# Use airport_heatmap exercise as base
# Use the Lat and Lng as locations and Humidity as the weight
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.
#### Narrow down the DataFrame to find your ideal weather condition. For example:
#### A max temperature lower than 80 degrees but higher than 70.
#### Wind speed less than 10 mph.
#### Zero cloudiness.
#### Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [8]:
Narrow_weath=CityWeath_df.loc[(CityWeath_df["Temperature"]<80) &
                              (CityWeath_df["Temperature"]>70) &
                              (CityWeath_df["Wind Speed"]<10) &
                              (CityWeath_df["Clouds"]==0) 
                             ]
Narrow_weath

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
358,352733,Marsá Maţrūḩ,EG,31.3525,27.2453,75.11,60,0,5.55
554,3359638,Walvis Bay,NaN,-22.9575,14.5053,74.35,35,0,5.75


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Add a "Hotel Name" column to the DataFrame
Hotel_df=Narrow_weath
Hotel_df["hotel name"]=""
Hotel_df

C:\Users\mosi\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,hotel name
358,352733,Marsá Maţrūḩ,EG,31.3525,27.2453,75.11,60,0,5.55,
554,3359638,Walvis Bay,NaN,-22.9575,14.5053,74.35,35,0,5.75,


In [10]:
# set up a parameters dictionary to use in search for hotels with 5000 meters
# use 01-Lesson-Plans/06-Python-APIs/3/Activities/02-Ins_Google_Places/Solved/Google_Places.ipynb as a guide
# Search responses from "https://developers.google.com/maps/documentation/places/web-service/search-nearby"
# hotels are under "lodging" https://developers.google.com/maps/documentation/places/web-service/supported_types
target_radius=5000
target_type="lodging"

params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
for index, row in Hotel_df.iterrows():
    params["location"]=f"{row['Lat']},{row['Lng']}"
    response = requests.get(base_url, params=params).json()
#     print(response)
    try:
        Hotel_df.loc[index,"hotel name"]=response["results"][0]["name"]
    except Exception as e:
        print(e)    
    

C:\Users\mosi\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [11]:
Hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,hotel name
358,352733,Marsá Maţrūḩ,EG,31.3525,27.2453,75.11,60,0,5.55,Beau Site
554,3359638,Walvis Bay,NaN,-22.9575,14.5053,74.35,35,0,5.75,Oyster Box Guesthouse


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in Hotel_df.iterrows()]
locations = Hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)

# Add layer
fig.add_layer(marker_layer)

# Display figure
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))